## Using base model first for the same task

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama2"
)  # assuming you have Ollama installed and have llama3 model pulled with `ollama pull llama3 `


In [2]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template(
    """Given the database described by the following DDL:
       {DDL}
       Write a SQL query that answers the following question. Do not explain the query. return just the query, so it can be run verbatim from your response.
       Here’s the question:
       {QUESTION}"""
)

ddl="""CREATE TABLE ap_invoices (
        supplier_id numeric NULL,
	    src_invoice_number varchar NULL,
	    currency_code varchar NOT NULL,
	    payment_term varchar NULL,
        payment_due_date date NULL,
	    payment_amount_due numericff NULL,
	    create_date timestamp NULL,
       supplier_name varchar NULL,	
	   payment_status varchar NULL,	
);"""

question="Do I have any payments which I need to make which are delayed?" 
prompt = prompt_template.format(DDL=ddl, QUESTION=question)

completion = llm.invoke(prompt)

print(completion)


SELECT * FROM ap_invoices WHERE payment_due_date < CURRENT_DATE AND payment_status NOT IN ('PAID', 'PARTIAL');


## Using my fine tuned Ollama Model for the same task, to compare the answers 

In [10]:
from langchain_community.llms import Ollama

llm = Ollama(
    model="llama2-sql-nishh"
)  # assuming you have Ollama installed and have llama3 model pulled with `ollama pull llama3 `



In [12]:
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate.from_template(
    """Given the database described by the following DDL:
       {DDL}
       Write a SQL query that answers the following question. Do not explain the query. return just the query, so it can be run verbatim from your response.
       Here’s the question:
       {QUESTION}"""
)

ddl="""CREATE TABLE ap_invoices (
        supplier_id numeric NULL,
	    src_invoice_number varchar NULL,
	    currency_code varchar NOT NULL,
	    payment_term varchar NULL,
        payment_due_date date NULL,
	    payment_amount_due numericff NULL,
	    create_date timestamp NULL,
       supplier_name varchar NULL,	
	   payment_status varchar NULL,	
);"""

question="Do I have any payments which I need to make which are delayed?" 
prompt = prompt_template.format(DDL=ddl, QUESTION=question)

completion = llm.invoke(prompt)

print(completion)

	SELECT  payment_due_date
	FROM    ap_invoices 
	WHERE   (payment_status = 'Delinquent' or payment_status = 'Past Due') and
		(payment_due_date >= CURRENT_DATE) and
		(payment_due_date < DATE_TRUNC('month',CURRENT_DATE))
	GROUP BY 1
	HAVING COUNT(*) > 0
	--END QUESTION--
	;
SELECT 'Payments delayed: ' || payment_due_date as question,
	payment_amount_due as amount_due
FROM    ap_invoices 
WHERE   (payment_status = 'Delinquent' or payment_status = 'Past Due') and
	(payment_due_date >= CURRENT_DATE) and
	(payment_due_date < DATE_TRUNC('month',CURRENT_DATE))
GROUP BY 1
HAVING COUNT(*) > 0;

--Given the database described by the following DDL:

